In [2]:
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

import simulator as sim

In [150]:
source = dict({'user_1': 150, 
               'user_2': 150})

# They are meant to be predicted using some model. i.e., linear regression or deep learning
class Service:
    def __init__(self, name_, svc_t_, m_tput_, plc_):
        self.name = name_
        self.service_time = svc_t_ # This should be some function(i.e., linear regression, fully connected layer) of something(i.e., load) eventually.
        self.max_tput = m_tput_
        self.placement = plc_
        
service_time = dict({"A":30.0,
                     "B":10.0})


# maximum amount of requests that can come through a node. Unlimited.
# Currently, they are set to be sum of total supply. It means no throughput constraint effectively.
# It could be useful later to limit the cross cluster routing.
node = dict({'A_start_1': 200, 
            'A_end_1': 200,

            'A_start_2': 200,
            'A_end_2': 200,

            'B_start_1': 200,
            'B_end_1': 200,

            'B_start_2': 200,
            'B_end_2': 200
            })

total_num_request = 0
for key, value in source.items():
    total_num_request += value
destination = dict({'dst': total_num_request}) # [user_1 supply] + [user_2 supply]

network_latency = dict({"region":80.0,
                        "rack":1.0})


# topology: A -> B -> C
arcs, cost = gp.multidict({
    # user_1: end user connected to cluster 1
    # user_2: end user connected to cluster 2
    ('user_1', 'A_start_1'): network_latency["rack"],
    ('user_2', 'A_start_2'): network_latency["rack"],
    ('user_1', 'A_start_2'): network_latency["region"],
    ('user_2', 'A_start_1'): network_latency["region"],
    
    # service time of A, it should be function of something.
    ('A_start_1', 'A_end_1'): service_time["A"],
    ('A_start_2', 'A_end_2'): service_time["A"],
    
    # network latency between A and B
    ('A_end_1', 'B_start_1'): network_latency["rack"],
    ('A_end_2', 'B_start_2'): network_latency["rack"],
    ('A_end_1', 'B_start_2'): network_latency["region"],
    ('A_end_2', 'B_start_1'): network_latency["region"],
    
    # service time of B, it should be function of something.
    ('B_start_1', 'B_end_1'): service_time["B"],
    ('B_start_2', 'B_end_2'): service_time["B"],
    
    ('B_end_1', 'dst'): 0.0,
    ('B_end_2', 'dst'): 0.0,
})

# # print(type(arcs))
# print("arcs: ", arcs)
# # print(dict(arcs).keys())
# # print(type(cost))
# print("cost: ", cost)

model = gp.Model('RequestRouting')
flow = model.addVars(arcs, obj=cost, name="flow")

# Constraint 1: source
src_keys = source.keys()
src_flow = model.addConstrs((gp.quicksum(flow.select(src, '*')) == source[src] for src in src_keys), name="source")

# Constraint 2: destination
dest_keys = destination.keys()
dst_flow = model.addConstrs((gp.quicksum(flow.select('*', dst)) == destination[dst] for dst in dest_keys), name="destination")

# Constraint 3(could be redundant): total source of source = total destination of destination
# sources = source.keys()
# dests = destination.keys()
# src_dst_flow = model.addConstrs(
#     (gp.quicksum(flow.select(s, '*') for s in sources) == 
#      gp.quicksum(flow.select('*', dst) for dst in dests)),
#     name="src_and_dest")

# Constraint 3: flow conservation
node_key = node.keys()
node_flow = model.addConstrs((gp.quicksum(flow.select(n_, '*')) == gp.quicksum(flow.select('*', n_)) for n_ in node_key), name="node")

# Constraint 4: max throughput of service
node_key = node.keys()
throughput = model.addConstrs((gp.quicksum(flow.select('*', n_)) <= node[n_] for n_ in node_key), name="service_capacity")

In [148]:
model.optimize()
if model.Status == GRB.INFEASIBLE:
    print("######################")
    print("### INFEASIBLE MODEL!")
    model.computeIIS()
    model.write("model.ilp")
    # print('\nThe following constraints and variables are in the IIS:')
    # for c in model.getConstrs():
    #     if c.IISConstr: print(f'\t{c.constrname}: {model.getRow(c)} {c.Sense} {c.RHS}')
    # for v in model.getVars():
    #     if v.IISLB: print(f'\t{v.varname} ≥ {v.LB}')
    #     if v.IISUB: print(f'\t{v.varname} ≤ {v.UB}')
    
# if model.Status == GRB.OPTIMAL:
#     solution = model.getAttr('X', flow)
#     for i, j in arcs:
#         if solution[i, j] > 0:
#             print('%s -> %s: %g' % (i, j, solution[i, j]))

request_flow = pd.DataFrame(columns=["From", "To", "Flow"])
for arc in arcs:
    if flow[arc].x > 1e-6:
        temp = pd.DataFrame({"From": [arc[0]], "To": [arc[1]], "Flow": [flow[arc].x]})
        request_flow = pd.concat([request_flow, temp], ignore_index=True)
request_flow
         
# Simple version of model relax
# if model.status == GRB.INFEASIBLE:
#     vars = model.getVars()
#     ubpen = [1.0]*model.numVars
#     model.feasRelax(1, False, vars, None, ubpen, None, None)
#     model.optimize()

# # Simple version of model relax
# if model.status == GRB.INFEASIBLE:
#     model.feasRelaxS(1, False, False, True)
#     model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 19 rows, 14 columns and 40 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 3e+02]

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.260000000e+04


,From,To,Flow
0,user_1,A_start_1,150.0
1,user_2,A_start_2,150.0
2,A_start_1,A_end_1,150.0
3,A_start_2,A_end_2,150.0
4,A_end_1,B_start_1,150.0
5,A_end_2,B_start_2,150.0
6,B_start_1,B_end_1,150.0
7,B_start_2,B_end_2,150.0
8,B_end_1,dst,150.0
9,B_end_2,dst,150.0
